In [23]:
import folium
import pandas as pd

In [24]:
df = pd.read_csv('../../data/processed/data_model_output.csv')

zip_centroids_file_path = '../../data/external/geometry/zip_code_centroids.csv'

C:\Users\mwe\AppData\Local\Temp\ipykernel_18108\4001696542.py:1: DtypeWarning: Columns (1,13,14,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,47,48,54,55,59,65,69,74,77,78,80,82,83,86,93,95,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,132,142,147,150,151,152,155,156,157,167,169,177,179,196,197,198,201,210,211,216,223,237,239,240,241,242,243,244,246,248,249,273,283,284,285,288,291,295,296,297,299,300,335,337,338,340,411,414,435) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../data/processed/data_model_output.csv')


In [25]:
df.head()

,unique_id,respondentid,is_completed,is_valid_record,date_completed,time_completed,is_pilot,is_self_administered,record_type_synthetic,access_mode,...,validation_num_errors_person,validation_num_errors_trip,validation_severity_person,validation_severity_trip,model_respondent_bool,weight_departing_and_arriving,weight_departing_only,weight_departing_only_model_respondents,weight_non_sas_departing_only,weight_departing_only_with_time_of_day
0,1,5473,True,True,2024-10-04,08:41:12,False,False,0,NaN,...,0,0,NaN,NaN,True,10.840259,11.525574,11.622594,19.179428,10.874504
1,2,5476,True,True,2024-10-04,08:40:04,False,False,0,1.0,...,0,0,NaN,NaN,True,6.487856,3.355533,3.346428,8.687559,6.502862
2,3,5489,True,True,2024-10-04,08:51:36,False,False,0,NaN,...,0,0,NaN,NaN,True,10.840259,11.525574,11.622594,19.179428,10.874504
3,4,5558,True,True,2024-10-04,10:32:58,False,False,0,1.0,...,0,0,NaN,NaN,True,6.487856,3.355533,3.346428,8.687559,6.502862
4,5,5593,True,True,2024-10-04,11:09:46,False,False,0,1.0,...,0,0,NaN,NaN,True,6.487856,3.355533,3.346428,8.687559,6.502862


In [26]:
df['party_size_flight'].value_counts()

party_size_flight
1    6291
2    2300
3     683
4     279
5     151
6      92
8      64
7      32
Name: count, dtype: int64

In [27]:
employee_df = df[(df['is_valid_record']==True) & (df['marketsegment_label']=='EMPLOYEE')]

In [28]:
employee_df['weight_departing_and_arriving'].sum()

np.float64(5998.5702102449895)

In [29]:
zip_weights = (
    employee_df.groupby('origin_zip')['weight_departing_and_arriving']
    .sum()
    .sort_values(ascending=False)
)

total_weight = zip_weights.sum()
top_25 = zip_weights.head(25)
top_25_percent = (top_25 / total_weight) * 100

# Combine into a DataFrame for readability
result_df = top_25.to_frame(name='total_weight')
result_df['percent'] = top_25_percent.round(2)

print(result_df)


            total_weight  percent
origin_zip                       
92101.0       474.108841     7.90
91910.0       362.850079     6.05
92104.0       342.053279     5.70
92103.0       226.743937     3.78
92102.0       194.560555     3.24
91950.0       188.555309     3.14
92111.0       183.415996     3.06
92109.0       166.072572     2.77
91911.0       155.633422     2.59
91977.0       155.232313     2.59
92115.0       153.497970     2.56
92117.0       153.063766     2.55
92113.0       153.015361     2.55
91945.0       149.145567     2.49
92114.0       138.706417     2.31
92154.0       135.687246     2.26
92116.0       131.784357     2.20
92106.0       124.445878     2.07
92110.0       107.534182     1.79
92108.0       101.881641     1.70
91915.0        97.562332     1.63
92071.0        97.562332     1.63
92126.0        93.209929     1.55
91942.0        93.209929     1.55
92105.0        92.727319     1.55


In [30]:
zip_weights = (
    employee_df.groupby('origin_pmsa_label')['weight_departing_and_arriving']
    .sum()
    .sort_values(ascending=False)
)

total_weight = zip_weights.sum()
top_25 = zip_weights.head(25)
top_25_percent = (top_25 / total_weight) * 100

# Combine into a DataFrame for readability
result_df = top_25.to_frame(name='total_weight')
result_df['percent'] = top_25_percent.round(2)

result_df

,total_weight,percent
origin_pmsa_label,,
CENTRAL,2191.759645,36.54
NORTH_CITY,1254.382222,20.91
SOUTH_SUBURBAN,1030.798756,17.18
EAST_SUBURBAN,783.484733,13.06
DOWNTOWN,298.529243,4.98
EXTERNAL,229.780894,3.83
NORTH_COUNTY_WEST,112.272386,1.87
NORTH_COUNTY_EAST,97.562332,1.63


In [31]:
employee_df['origin_pmsa_label'].value_counts()

origin_pmsa_label
CENTRAL              249
NORTH_CITY           148
SOUTH_SUBURBAN       109
EAST_SUBURBAN         76
DOWNTOWN              34
EXTERNAL              22
NORTH_COUNTY_WEST     12
NORTH_COUNTY_EAST      9
Name: count, dtype: int64

In [32]:
zip_to_coordinates = pd.read_csv(zip_centroids_file_path)

In [33]:
zip_to_coordinates.head()

,OBJECTID,STD_ZIP5,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE,LATITUDE,LONGITUDE,LAT,LON,x,y
0,1,601,ADJUNTAS,PR,18.163996,-66.719018,18.16400,-66.71902,-7.427127e+06,2.056753e+06
1,2,602,AGUADA,PR,18.385289,-67.183839,18.38529,-67.18384,-7.478871e+06,2.082696e+06
2,3,603,AGUADILLA,PR,18.450964,-67.144917,18.45096,-67.14492,-7.474538e+06,2.090401e+06
3,4,606,MARICAO,PR,18.191914,-66.983219,18.19191,-66.98322,-7.456537e+06,2.060024e+06
4,5,610,ANASCO,PR,18.284550,-67.146862,18.28455,-67.14686,-7.474754e+06,2.070882e+06


In [34]:
# Clean and group data
employee_df['origin_zip'] = employee_df['origin_zip'].astype(str).str.split('.').str[0].astype(int)
employee_df = employee_df.groupby('origin_zip', as_index=False)['weight_departing_and_arriving'].sum()

# Clean coordinates
zip_to_coordinates['STD_ZIP5'] = zip_to_coordinates['STD_ZIP5'].astype(str).str.split('.').str[0].astype(int)

# Merge
merged_df = pd.merge(
    employee_df,
    zip_to_coordinates[['STD_ZIP5', 'LATITUDE', 'LONGITUDE']],
    left_on='origin_zip',
    right_on='STD_ZIP5',
    how='left'
)
merged_df = merged_df.dropna(subset=['LATITUDE', 'LONGITUDE'])

# Plot
m = folium.Map(location=[32.7157, -117.1611], zoom_start=10)

for _, row in merged_df.iterrows():
    folium.CircleMarker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        radius=row['weight_departing_and_arriving'] / 20,
        color='black',           # border color
        weight=1.5,              # border thickness
        fill=True,
        fill_color='blue',       # circle fill color
        fill_opacity=1,
        popup=f"{row['origin_zip']}: {row['weight_departing_and_arriving']}"
    ).add_to(m)
m


C:\Users\mwe\AppData\Local\Temp\ipykernel_18108\3945576779.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  employee_df['origin_zip'] = employee_df['origin_zip'].astype(str).str.split('.').str[0].astype(int)


In [35]:
merged_df.head()

,origin_zip,weight_departing_and_arriving,STD_ZIP5,LATITUDE,LONGITUDE
1,22044,10.840259,22044.0,38.866758,-77.150331
2,22046,10.840259,22046.0,38.885517,-77.180792
6,22504,10.840259,22504.0,37.765761,-76.715067
7,91709,10.840259,91709.0,33.974234,-117.722517
8,91711,10.840259,91711.0,34.109043,-117.720282


In [36]:
employee_df['origin_zip'].head()

0    22000
1    22044
2    22046
3    22056
4    22170
Name: origin_zip, dtype: int64